In [168]:
#import pandas as pd
import numpy as np
import sys
from multiprocessing import Process, Pipe
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from random import shuffle
from os import cpu_count

In [2]:
try:
    with open('./input/inputs.npy',"rb") as f:
        inputs = np.load(f)
    with open('./input/outputs.npy',"rb") as f:
        outputs = np.load(f)
        
except:
    data = np.genfromtxt('./input/train.csv', delimiter=',',skip_header=1,dtype=np.uint8)
    inputs = data[:,1:]
    outputs = data[:,0]
    np.save('./input/inputs.npy',inputs)
    np.save('./input/outputs.npy',outputs)
print(f'Input:\nShape:{inputs.shape}\n{inputs[:5,:5]}\n')
print(f'Output:\nShape:{outputs.shape}\n{outputs}\n')

Input:
Shape:(42000, 784)
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Output:
Shape:(42000,)
[1 0 1 ... 7 6 9]



In [3]:
def onehot(ARRAY):
    return np.eye(10)[ARRAY].astype(np.uint8).T

In [4]:
inputs = normalize(inputs)
X_train, X_test, y_train, y_test = train_test_split(inputs,outputs,test_size = 0.95)

X_train = X_train.T
y_train_hot = onehot(y_train)

l_dim = np.array([X_train.shape[0],485,299,185,114,71,44,27,17,10], dtype = np.uint16)
w = [int]*(len(l_dim)+1)
b = [int]*(len(l_dim)+1)

In [278]:
def sigmoid(X):
    return np.exp(X)/(1 + np.exp(X))

def sigprime(X):
    return np.exp(X)/np.power(1 + np.exp(X),2)

def initialize_wb(LAYERS):
    for i in range(0,len(LAYERS)-1):
        w[i] = np.array(np.random.random((LAYERS[i+1],LAYERS[i])), dtype=np.float128)
        b[i]  = np.zeros((LAYERS[i+1],1), dtype=np.float16)
    return w, b

def feedforward(W,X,B):
    return normalize(sigmoid(np.dot(W,X) + B))

def costfunction(PRED,TRUE):
    return 1/2*np.sum(np.power(np.subtract(PRED,TRUE),2))

def backprop(WA, WB, LA, LB, LB_TRUE, BB): #Layer A calculates Layer B
    # NewWeight = OldWeight - Strength*dC/dw
    S = 1
    z = np.dot(WB,LA) + BB
    #dC/dw
    derivative = 2*(LB - LB_TRUE)*sigprime(z)*LA
    neweights = WB - S*derivative
    print(WA.shape)
    return newweights

In [281]:
layers[8].shape

(10, 2100)

In [280]:
#np.average((np.dot(w[7],layers[6]) - y_train),axis=1).shape
#print(w[8].shape)
#(w[8] - np.average((np.dot(w[7],layers[6]) - y_train),axis=1))
#backprop(w[7],w[8],layers[7],layers[8],y_train,b[8])

ValueError: operands could not be broadcast together with shapes (10,2100) (17,2100) 

In [217]:
for i in range(0,17):  
    print(w[8][:,i])
    newweights = w[8][:,i] - np.average(sigmoid(np.dot(w[8],layers[7])),axis=1)

newweights

[0.50012567 0.90906139 0.68327564 0.74803134 0.99534114 0.22028091
 0.47513246 0.40736243 0.49415167 0.25388683]
[0.79637706 0.02299958 0.66891725 0.30704816 0.68049157 0.14000174
 0.48736809 0.20919174 0.33618122 0.73571788]
[0.08806844 0.33915092 0.18988858 0.82931465 0.48607477 0.86454918
 0.57602141 0.94428312 0.27769796 0.26633072]
[0.83233463 0.31266737 0.59785971 0.51607987 0.74082653 0.75537495
 0.06651911 0.31722249 0.36726151 0.1841184 ]
[0.6227641  0.18150732 0.404347   0.17862747 0.53445847 0.78281584
 0.26511397 0.47032699 0.53797666 0.27489798]
[0.99806897 0.02274723 0.05306573 0.54251795 0.72547596 0.82045725
 0.86948974 0.79745073 0.74657598 0.94218762]
[0.00297068 0.24530665 0.06952082 0.16911954 0.21831272 0.79571182
 0.06294608 0.32522796 0.65852453 0.70271382]
[0.43476152 0.64681979 0.49735545 0.74236573 0.58188057 0.75486577
 0.42648977 0.57683366 0.55268934 0.60621237]
[0.11256105 0.50399949 0.34202843 0.31286847 0.54727588 0.59948736
 0.25994255 0.61784567 0.0153

array([-0.34267532, -0.28363217,  0.08170846,  0.05236307, -0.02207727,
       -0.30352922,  0.22884788,  0.03468156, -0.44139749, -0.51258936],
      dtype=float128)

In [216]:
w, b = initialize_wb(l_dim)
layers = []
layer = X_train
for i in range(0,len(l_dim)-1):
    sys.stdout.write(f'{i}\r')
    newlayer = feedforward(w[i],layer,b[i])
    print(newlayer.shape)
    layers.append(newlayer)
    layer = newlayer
print(costfunction(layer[8],y_train))

(485, 2100)
(299, 2100)
(185, 2100)
(114, 2100)
(71, 2100)
(44, 2100)
(27, 2100)
(17, 2100)
(10, 2100)
28319.5725712473


In [37]:
print(f'{layer[9].shape}')
print(f'{y_train.shape}')

(2100,)
(2100,)


In [14]:
for l in layers:
    print(l.shape)

(485, 2100)
(299, 2100)
(185, 2100)
(114, 2100)
(71, 2100)
(44, 2100)
(27, 2100)
(17, 2100)
(10, 2100)


In [17]:
backprop(w[7],layer[7],layer[8],y_train)

ValueError: shapes (17,27) and (2100,) not aligned: 27 (dim 1) != 2100 (dim 0)

In [17]:
backprop(w[])

[array([[0.02188446, 0.02186296, 0.02197822, ..., 0.02198359, 0.02190449,
         0.02165446],
        [0.02187859, 0.02184836, 0.02192142, ..., 0.02191524, 0.02189598,
         0.02168718],
        [0.02175002, 0.02175859, 0.02199114, ..., 0.02199479, 0.02186999,
         0.02162903],
        ...,
        [0.0218813 , 0.02185595, 0.02195276, ..., 0.02194159, 0.02187636,
         0.02169529],
        [0.02178212, 0.02175541, 0.02193507, ..., 0.02194156, 0.02190111,
         0.02165755],
        [0.021805  , 0.02173554, 0.02190916, ..., 0.02189457, 0.02188438,
         0.02179981]]),
 array([[0.02182216, 0.02182137, 0.02182489, ..., 0.02182486, 0.02182368,
         0.02181951],
        [0.02182212, 0.02182142, 0.02182478, ..., 0.02182473, 0.02182359,
         0.02181959],
        [0.02182216, 0.02182142, 0.02182503, ..., 0.021825  , 0.02182372,
         0.02181933],
        ...,
        [0.0218221 , 0.02182149, 0.02182433, ..., 0.0218243 , 0.02182331,
         0.02181989],
        [0.0